自然语言处理中更常用的是RNN:Recurrent Neural Network,能够更好地表达上下文信息
RNN是在自然语言处理中非常标配的一个网络，在序列标注/命名体识别/Seq2seq灯场景中都有应用
RNN的对应公式:P(w1,w2...wm)=(连乘(i=1,m))P(wi|w1,...wi-1)
这里的每个词对应的概率依赖于前面所有的词对应的概率，与

In [4]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode, modify by wmathor
'''
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
#word_list=['i','like','dog','i','love','coffee','i','hate','milk']
vocab = list(set(word_list))
#vocab=['hate', 'love', 'milk', 'like', 'dog', 'coffee', 'i']
word2idx = {w: i for i, w in enumerate(vocab)}
#word2idx = {'hate': 0, 'love': 1, 'milk': 2, 'like': 3, 'dog': 4, 'coffee': 5, 'i': 6}
#!!!注意!!!每次word2idx与对应的单词编码的结果可能会不一样
print('word2idx = ***')
print(word2idx)
idx2word = {i: w for i, w in enumerate(vocab)}
#idx2word = {0: 'hate', 1: 'love', 2: 'milk', 3: 'like', 4: 'dog', 5: 'coffee', 6: 'i'}
print('idx2word = ***')
print(idx2word)
n_class = len(vocab)
#n_class = 7,因为总共有对应的7个单词
# TextRNN Parameter
batch_size = 2
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

def make_data(sentences):
    print('make_data')
    input_batch = []
    target_batch = []

    for sen in sentences:
        #print('sen = ***')
        #print(sen)
        word = sen.split()
        print('word = ***')
        print(word)
        input = [word2idx[n] for n in word[:-1]]
        #print('input = ***')
        #print(input)
        target = word2idx[word[-1]]
        #将前两个单词作为对应的input，最后一个单词
        #作为目标对应的target的值，比如'i','like','dog'
        #input=[6,3]为'i','like'对应的值，'dog'=[4]为
        #target对应的值
        #print('target = ***')
        #print(target)
        input_batch.append(np.eye(n_class)[input])
        #这里面n_class的对应值为7，numpy.eye()为生成对角矩阵
        #对应的n_class值为7的时候为长度为7的对角矩阵，这里的
        #input选择哪个的时候对应的值就为哪个，所以本质上是
        #将对应的input的值进行one-hot的编码
        print('input_batch = ***')
        print(input_batch)
        target_batch.append(target)
    return input_batch, target_batch

input_batch, target_batch = make_data(sentences)
print('----------------------------------------------------------------------------')
print('input_batch = !!!')
print(input_batch)
print('target_batch = !!!')
print(target_batch)
input_batch, target_batch = torch.Tensor(input_batch), torch.LongTensor(target_batch)
print('############################################################################')
print('input_batch.shape = ***')
print(input_batch.shape)
print('target_batch.shape = ***')
print(target_batch.shape)
'''
input_batch为one-hot对应的向量组成的相应的数组
target_batch为对应target的值
'''
dataset = Data.TensorDataset(input_batch, target_batch)
#TensorDataset:对给定的tensor数据(样本和标签),将它们包装成dataset
#注意如果是numpy的array，或者Pandas的DataFrame需要先转换成Tensor

#将对应的tensor组装成一个input_batch和target_batch构成的一个整体
print('dataset 0 = ***')
print(dataset[0])
#比如dataset[0]组成的对应的值为
#(tensor([[0., 0., 0., 0., 0., 1., 0.],
#        [0., 1., 0., 0., 0., 0., 0.]]), tensor(6))
loader = Data.DataLoader(dataset, batch_size, True)
print('----------------loader = --------------------------')
for  img,label  in  loader:
    print('img.shape = ***')
    print(img.shape)
    print('label.shape = ***')
    print(label.shape)

word2idx = ***
{'dog': 0, 'hate': 1, 'love': 2, 'milk': 3, 'like': 4, 'coffee': 5, 'i': 6}
idx2word = ***
{0: 'dog', 1: 'hate', 2: 'love', 3: 'milk', 4: 'like', 5: 'coffee', 6: 'i'}
make_data
word = ***
['i', 'like', 'dog']
input_batch = ***
[array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.]])]
word = ***
['i', 'love', 'coffee']
input_batch = ***
[array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.]])]
word = ***
['i', 'hate', 'milk']
input_batch = ***
[array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0.]])]
----------------------------------------------------------------------------
input_batch = !!!
[array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.]]), array([[0., 0., 0., 0.,

batch_size的对应值为2，
dataset为刚才写的input组成的one-hot编码以及target_batch的tensor组成的
对应的结果值
DataLoader本质上是一个iterable,与python内置的list类似，并利用多进程
来加速batch data，使用yield使用有限的内存

DataLoader是一个高效、简洁、直观的网络输入数据结果，
使用pytorch将数据加载到模型一般的操作顺序如下：
1.创建一个Dataset对象
2.创建一个DataLoader对象
3.循环这个DataLoader对象，将img，label加载到模型中进行训练

比如本例子之中对应的word2idx的对应值如下:
{'hate':0,'like':1,'coffee':2,'milk':3,'love':4,'i':5,'dog':6}
那么此时对应的dataset的内容为
[([5,1],[6]),([5,4],[2]),([5,0],[3])]
经过DataLoader之后，对应的loader的内容为
[([5,4],[5,0]),([2],[3]),([5,1]),([6])]

In [9]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        print('__init__ TextRnn')
        print('n_class = %d'%n_class)
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        #input_size:输入特征的维度，hidden_size:隐藏层神经元的个数
        #这里体现了使用RNN对文本进行操作分类
        self.fc = nn.Linear(n_hidden, n_class)
        #乘上对应的n_hidden*n_class的对应的矩阵

    def forward(self, hidden, X):
        print('hidden.shape = ***')
        print(hidden.shape)
        print('X.shape = ***')
        print(X.shape)
        print('forward = ***')
        # X: [batch_size, n_step, n_class]
        print('before transpose')
        print('X.shape = ***')
        print(X.shape)
        print('type X = ***')
        print(type(X))
        #变换之前X的对应的矩阵为
        #[[[0,0,0,0,1,0,0],
        #  [0,0,0,0,0,0,1]],
        #  [0,0,0,0,1,0,0],
        #  [0,0,0,1,0,0,0]]]
        X = X.transpose(0, 1) # X : [n_step, batch_size, n_class]
        #pytorch中的transpose方法的作用是交换矩阵的两个维度
        
        #transpose中只有两个参数，torch.transpose中有三个参数
        print('after transpose')
        print('X.shape = ***')
        print(X.shape)
        #变换之后X对应的矩阵为
        #[[[0,0,0,0,1,0,0],
        #  [0,0,0,0,1,0,0],
        #  [0,0,0,0,0,0,1],
        #  [0,0,0,1,0,0,0]]]
        print('hidden = ***')
        print(hidden)
        print('self.rnn = ***')
        out, hidden = self.rnn(X, hidden)
        #对应的self.rnn,这里使用nn.rnn(input_size,hidden_size)
        #其中input_size为输入特征的维度，hidden_size为隐藏层神经元的
        #对应的个数，或者也叫输出的维度
        
        #比如这里定义了rnn_layer = nn.RNN(input_size=1027,hidden_size=256)
        #然后这里面使用self.runn(X,5)(隐藏层对应的个数为5)
        #输入的X为对应的([35,2,1027])的对应的矩阵，输出的Y为对应的
        #[35,2,256]的对应的矩阵，
        
        # out : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        print('!!!out.shape = !!!')
        print(out.shape)
        print('!!!hidden.shape = !!!')
        print(hidden.shape)
        #原先的out对应的矩阵内容为
        #out = tensor([[[-0.3484, -0.1260, -0.2226, -0.0077, -0.4533],
        #[-0.3484, -0.1260, -0.2226, -0.0077, -0.4533]],
        #[[-0.5938,  0.3629, -0.2776,  0.2037, -0.1790],
        #[-0.7574, -0.4174, -0.5710,  0.1534,  0.0740]]],grad_fn=<SelectBackward>)
        
        out = out[0] # [batch_size, num_directions(=1) * n_hidden] ⭐
        #经过out[-1]的结果之后，对应的内容为
        #out = tensor([[[-0.5938,  0.3629, -0.2776,  0.2037, -0.1790],
        #[-0.7574, -0.4174, -0.5710,  0.1534,  0.0740]]],grad_fn=<SelectBackward>)
        #因为总共数组里存在两个对应的内容，所以[-1]选取的是最后面的那个
        print('out = ###')
        print(out)
        print('out.shape = ###')
        print(out.shape)
        model = self.fc(out)
        #这里的定义为self.fc = nn.Linear(n_hidden,n_class)
        print('n_hidden = %d,n_class = %d'%(n_hidden,n_class))
        print('model = ###')
        print(model.shape)
        #这里面的内容为n_hidden = 5,n_class = 7,本身输入的内容为
        #2*5的对应的内容，所以输出的内容为2*7的对应的内容，因为总共有
        #7个类别，需要对应到7个类别
        return model

model = TextRNN()
criterion = nn.CrossEntropyLoss()
#分类问题的处理一般都使用交叉熵损失函数
#对应的公式为H(x)=-(i=1~n的和)P(xi)log(P(xi))
#相对熵(KL散度)概念:DKL(p||q) = (i=1~n)p(xi)log(p(xi)/q(xi)),
#其中这里的q(xi)代表的含义为原先预测的对应的概率
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(5):
    print('--------------------------------------------------------------------------------------------------------')
    print('epoch = %d'%epoch)
    for x, y in loader:
        print('x = ***')
        print(x.shape)
        print('y = ***')
        print(y.shape)
        # hidden : [num_layers * num_directions, batch, hidden_size]
        print('n_hidden = ***')
        print(n_hidden)
        hidden = torch.zeros(1, x.shape[0], n_hidden)
        # x : [batch_size, n_step, n_class]
        #torch.zeros(*size,*,out=None,dtype=None,layout=torch.strided,
        #device=None,requires_grad=False)
        #这里的out(Tensor)代表着对应的the  output tensor,
        #x.shape[0] = 2,n_hidden = 5,对应的矩阵为1*2*5
        #!!!rnn当中的隐藏层的对应输入必须为零矩阵!!!
        
        #实际形成的是一个1*2*5对应的相应的矩阵
        print('hidden = ***')
        print(hidden.shape)
        #hidden = 5,为上面的对应的隐藏单元
        
        #这里面调用model里面对应的forward()的相应的函数
        print('x.shape = ***')
        print(x.shape)
        pred = model(hidden, x)
        #之前的model = TextRNN()只是定义了相应的参数，
        #使用pred = model(hidden,x)直接进入相应的forward函数之中进行运行
        print('pred = ***')
        print(pred.shape)
        print('y = ***')
        print(y.shape)
        # pred : [batch_size, n_class], y : [batch_size] (LongTensor, not one-hot)
        # pred为输出的对应的2*7的矩阵
        print('pred = ###')
        print(pred)
        print('y = ###')
        print(y)
        print('!!!before loss!!!')
        loss = criterion(pred, y)
        print('loss = %.6f'%loss)
        #这里在求criterion的时候，pred=(2,7),y=(2)
        #前面定义了criterion = nn.CrossEntropyLoss()
        #维度前面的维度可以不同，后面的维度必须相同
        
        #所以这里为pred与y的交叉熵损失函数
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        optimizer.zero_grad()
        print('loss = ***')
        print(loss)
        loss.backward()
        optimizer.step() 

__init__ TextRnn
n_class = 7
--------------------------------------------------------------------------------------------------------
epoch = 0
x = ***
torch.Size([2, 2, 7])
y = ***
torch.Size([2])
n_hidden = ***
5
hidden = ***
torch.Size([1, 2, 5])
x.shape = ***
torch.Size([2, 2, 7])
hidden.shape = ***
torch.Size([1, 2, 5])
X.shape = ***
torch.Size([2, 2, 7])
forward = ***
before transpose
X.shape = ***
torch.Size([2, 2, 7])
type X = ***
<class 'torch.Tensor'>
after transpose
X.shape = ***
torch.Size([2, 2, 7])
hidden = ***
tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])
self.rnn = ***
!!!out.shape = !!!
torch.Size([2, 2, 5])
!!!hidden.shape = !!!
torch.Size([1, 2, 5])
out = ###
tensor([[ 0.2114, -0.2030,  0.6489,  0.7838, -0.4902],
        [ 0.2114, -0.2030,  0.6489,  0.7838, -0.4902]],
       grad_fn=<SelectBackward>)
out.shape = ###
torch.Size([2, 5])
n_hidden = 5,n_class = 7
model = ###
torch.Size([2, 7])
pred = ***
torch.Size([2, 7])
y = ***
torch.Size([2])
pred =

In [6]:
input = [sen.split()[:2] for sen in sentences]
#input为截取的前面的两个单词作为相应的输入的内容
# Predict,!!!
print('input = ***')
print(input)
hidden = torch.zeros(1, len(input), n_hidden)
print('hidden = ***')
print(hidden.shape)
#hidden对应的torch的大小为([1,3,5])01
print('input_batch = ###')
print(input_batch)
#这里面的model相当于前面训练过的对应的model的内容
predict1 = model(hidden, input_batch)
print('predict1 = ***')
print(predict1)
#得到对应的不同类别的预测的概率值
predict2 = predict1.data.max(1,keepdim=True)
#提取出着所有单词当中出现最大的概率，提取出来的
#tensor有两个部分，第一个部分为value是对应的概率
#第二个部分为indices为对应的概率相应的具体的数值
print('predict2 = ***')
print(predict2)
predict = predict2[1]
print('predict = ***')
print(predict)
#predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
#对应的预测种类的结果
print([sen.split()[:2] for sen in sentences], '->', [idx2word[n.item()] for n in predict.squeeze()])


input = ***
[['i', 'like'], ['i', 'love'], ['i', 'hate']]
hidden = ***
torch.Size([1, 3, 5])
input_batch = ###
tensor([[[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0., 0., 0.]]])
hidden.shape = ***
torch.Size([1, 3, 5])
X.shape = ***
torch.Size([3, 2, 7])
forward = ***
before transpose
X = ***
tensor([[[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0., 0., 0.]]])
type X = ***
<class 'torch.Tensor'>
after transpose
X = ***
tensor([[[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.

程序运行的过程解析：首先
x = tensor([[[0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.]]])
(x为起始的两个对应的句子)
y = tensor([2,0])
(y为对应的两个答案内容的标签)

接着对应的hidden矩阵的内容为
hidden = tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

optimizer的step为什么不能放在min-batch的循环之外，optimizer.step和loss.backward的区别
1.明确optimizer优化器的作用:根据网络反向传播的梯度信息更新网络的参数，以起到降低loss函数计算值的作用
    1.优化器需要知道当前网络或者别的模型的参数空间
    2.优化器需要知道反向传播的梯度信息
    optimizer更新参数空间需要基于反向梯度，因此当调用optimizer.step()的时候应当是loss.backward()的时候，所以经常是loss.backward()后面跟上optimizer.step()函数
    

rnn前向神经模型对应的网站https://blog.csdn.net/weixin_42792500/article/details/81254313
可以看出隐藏层输入必须为零矩阵，所以